# Lesson 3: Training Models in Low Carbon Regions

* Import libraries to train a model locally.

In [1]:
import numpy as np
from sklearn.datasets import make_blobs
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

2024-07-03 14:50:49.731836: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-03 14:50:49.850828: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-07-03 14:50:49.850858: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-07-03 14:50:50.704540: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

* Create a dataset.

This is an example from the [Advanced Learning Algorithms](https://www.coursera.org/learn/advanced-learning-algorithms) course.

In [2]:
classes = 4
m = 100
centers = [[-5, 2], [-2, -2], [1, 2], [5, -2]]
std = 1.0
X_train, y_train = make_blobs(
    n_samples=m, 
    centers=centers, 
    cluster_std=std,
    random_state=30)

* Create the model.

In [3]:
# Create the model
model = Sequential(
    [
        Dense(2, activation = 'relu',   name = "L1"),
        Dense(4, activation = 'linear', name = "L2")
    ]
)

2024-07-03 14:50:51.899321: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-07-03 14:50:51.899349: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-07-03 14:50:51.899393: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (5ca142449ca6): /proc/driver/nvidia/version does not exist
2024-07-03 14:50:51.899620: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.01),
)

* Train the model.

In [5]:
model.fit(
    X_train,y_train,
    epochs=200
)

Epoch 1/200
4/4 [==============================] - 0s 3ms/step - loss: 2.1115
Epoch 2/200
4/4 [==============================] - 0s 1ms/step - loss: 1.8805
Epoch 3/200
4/4 [==============================] - 0s 1ms/step - loss: 1.6915
Epoch 4/200
4/4 [==============================] - 0s 1ms/step - loss: 1.5204
Epoch 5/200
4/4 [==============================] - 0s 1ms/step - loss: 1.3561
Epoch 6/200
4/4 [==============================] - 0s 1ms/step - loss: 1.2254
Epoch 7/200
4/4 [==============================] - 0s 1ms/step - loss: 1.1251
Epoch 8/200
4/4 [==============================] - 0s 2ms/step - loss: 1.0579
Epoch 9/200
4/4 [==============================] - 0s 1ms/step - loss: 1.0002
Epoch 10/200
4/4 [==============================] - 0s 1ms/step - loss: 0.9518
Epoch 11/200
4/4 [==============================] - 0s 1ms/step - loss: 0.9124
Epoch 12/200
4/4 [==============================] - 0s 1ms/step - loss: 0.8798
Epoch 13/200
4/4 [==============================] - 0s 1ms/st

4/4 [==============================] - 0s 1ms/step - loss: 0.2121
Epoch 105/200
4/4 [==============================] - 0s 2ms/step - loss: 0.2025
Epoch 106/200
4/4 [==============================] - 0s 1ms/step - loss: 0.1938
Epoch 107/200
4/4 [==============================] - 0s 1ms/step - loss: 0.1864
Epoch 108/200
4/4 [==============================] - 0s 1ms/step - loss: 0.1800
Epoch 109/200
4/4 [==============================] - 0s 1ms/step - loss: 0.1744
Epoch 110/200
4/4 [==============================] - 0s 1ms/step - loss: 0.1688
Epoch 111/200
4/4 [==============================] - 0s 1ms/step - loss: 0.1646
Epoch 112/200
4/4 [==============================] - 0s 1ms/step - loss: 0.1604
Epoch 113/200
4/4 [==============================] - 0s 1ms/step - loss: 0.1572
Epoch 114/200
4/4 [==============================] - 0s 1ms/step - loss: 0.1532
Epoch 115/200
4/4 [==============================] - 0s 1ms/step - loss: 0.1504
Epoch 116/200
4/4 [==============================] - 0

### Running on Google Cloud

We will use Google Cloud's ML platform, Vertex AI. To run this code on Vertex AI, we first need to:
1. Import and initialize the Vertex AI Python SDK
2. Write the ML training code to a file
3. Configure and submit a training job that runs our training code.

* Step 1. Initialize Vertex AI.

In [6]:
from helper import authenticate
CREDENTIALS, PROJECT_ID = authenticate()

In [7]:
from google.cloud import aiplatform

In [8]:
aiplatform.init(project=PROJECT_ID,
                credentials=CREDENTIALS,
                )

* Step 2. Write the ML training code to a file.

In [9]:
%%writefile task.py

# import libraries
import numpy as np
from sklearn.datasets import make_blobs
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Create dataset
classes = 4
m = 100
centers = [[-5, 2], [-2, -2], [1, 2], [5, -2]]
std = 1.0
X_train, y_train = make_blobs(
    n_samples=m, 
    centers=centers, 
    cluster_std=std,
    random_state=30)

# Create the model
model = Sequential(
    [
        Dense(2, activation = 'relu',   name = "L1"),
        Dense(4, activation = 'linear', name = "L2")
    ]
)

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.01),
)

# Train
model.fit(
    X_train,y_train,
    epochs=200
)

Overwriting task.py


In [10]:
ls

L3_Training_Models.ipynb*  __pycache__/  helper.py*  task.py*


In [11]:
cat task.py


# import libraries
import numpy as np
from sklearn.datasets import make_blobs
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Create dataset
classes = 4
m = 100
centers = [[-5, 2], [-2, -2], [1, 2], [5, -2]]
std = 1.0
X_train, y_train = make_blobs(
    n_samples=m, 
    centers=centers, 
    cluster_std=std,
    random_state=30)

# Create the model
model = Sequential(
    [
        Dense(2, activation = 'relu',   name = "L1"),
        Dense(4, activation = 'linear', name = "L2")
    ]
)

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.01),
)

# Train
model.fit(
    X_train,y_train,
    epochs=200
)


* Define the Custom Trainin Job.

> Note: Check here to find more information about [Prebuilt containers for custom training](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)

> Note: Check here to find more information about [Carbon free energy for Google Cloud regions](https://cloud.google.com/sustainability/region-carbon)

In [12]:
# Choose a location (with carbon free energy available for this example)
REGION = 'us-central1'

* Create a store bucket.

In [13]:
import random
import string

In [14]:
def generate_uuid(length: int = 8) -> str:
    return "".join(
        random.choices(
            string.ascii_lowercase + string.digits, 
            k=length))

UUID = generate_uuid()

In [15]:
# The unique identifier
UUID

'jzqr65g3'

In [16]:
from google.cloud import storage

In [17]:
storage_client = storage.Client(project = PROJECT_ID,
                                credentials = CREDENTIALS)

In [18]:
bucket_name = f'carbon-course-bucket-{UUID}'

In [19]:
bucket_name

'carbon-course-bucket-jzqr65g3'

In [20]:
bucket = storage_client.bucket(bucket_name)

In [21]:
bucket.create(location=REGION)

In [22]:
# Create the CustomTrainingJob
job = aiplatform.CustomTrainingJob(
    display_name='dlai-course-example',
    script_path='task.py',
    container_uri='us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-14.py310:latest',
    staging_bucket=f'gs://{bucket_name}',
    location=REGION,
)

In [23]:
model = job.run()

Training script copied to:
gs://carbon-course-bucket-jzqr65g3/aiplatform-2024-07-03-14-50-59.129-aiplatform_custom_trainer_script-0.1.tar.gz.
Training Output directory:
gs://carbon-course-bucket-jzqr65g3/aiplatform-custom-training-2024-07-03-14-50-59.129
View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/3727316689851777024?project=362161713137
View backing custom job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/1619438550195896320?project=362161713137
CustomTrainingJob projects/362161713137/locations/us-central1/trainingPipelines/3727316689851777024 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomTrainingJob projects/362161713137/locations/us-central1/trainingPipelines/3727316689851777024 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomTrainingJob projects/362161713137/locations/us-central1/trainingPipelines/3727316689851777024 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomTrainingJ

In [24]:
# Delete the bucket to clean up resources
bucket.delete(force=True)